Preparación de los datos
Este es el paso más crucial para cualquier modelo de machine learning. Aquí debemos:

Tratar los datos faltantes: Eliminarlos o imputarlos.
Eliminar outliers o aplicar técnicas de transformación.
Normalización o escalado de datos: Esto es particularmente importante para modelos como Regresión Lineal o K-Nearest Neighbors (KNN).
One-Hot Encoding para variables categóricas: Transformar variables categóricas en variables binarias para que puedan ser utilizadas por los modelos.

Dividir los datos
Es fundamental dividir los datos en train y test. Vamos a utilizar 80% para entrenar y 20% para testear.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ML_Model").getOrCreate()

# Leer los datos del archivo CSV y cargarlos en un DataFrame
df = spark.read.csv("/FileStore/tables/ecommerce_data_cleaned.csv", header=True, inferSchema=True)

# Crear la columna 'label' desde Revenue
df_ml = df.withColumn("label", col("Revenue"))

# Mostrar algunas filas
df_ml.show(5)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+-----+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|           Revenue|Month|             label|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+-----+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|15.299999999999999|   12|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|             20.34|   12|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|              22.0|   12|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   1785

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Definir las etapas del Pipeline
indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
encoder = OneHotEncoder(inputCol="CountryIndex", outputCol="CountryVec")
assembler = VectorAssembler(inputCols=["Quantity", "UnitPrice", "Month", "CountryVec"], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# Crear el Pipeline con todas las etapas
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler])

# Aplicar el pipeline a df_ml (que contiene 'label')
df_prepared = pipeline.fit(df_ml).transform(df_ml)

# Mostrar las características y etiquetas generadas
df_prepared.select("features", "label").show(5)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

+--------------------+------------------+
|            features|             label|
+--------------------+------------------+
|(40,[0,1,2,3],[6....|15.299999999999999|
|(40,[0,1,2,3],[6....|             20.34|
|(40,[0,1,2,3],[8....|              22.0|
|(40,[0,1,2,3],[6....|             20.34|
|(40,[0,1,2,3],[6....|             20.34|
+--------------------+------------------+
only showing top 5 rows



In [0]:

# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = df_prepared.sample(fraction=0.5, seed=42).randomSplit([0.8, 0.2], seed=42)

# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {train_data.count()}")
print(f"Tamaño del conjunto de prueba: {test_data.count()}")


Tamaño del conjunto de entrenamiento: 210990
Tamaño del conjunto de prueba: 52468


In [0]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Definir el GBTRegressor
gbt = GBTRegressor(featuresCol="scaledFeatures", labelCol="label")

# Definir el evaluador (usando RMSE como métrica de evaluación)
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Crear un grid de hiperparámetros para ajustar
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 7]) \
    .addGrid(gbt.maxIter, [10]) \
    .addGrid(gbt.stepSize, [0.1]) \
    .build()

# Configurar validación cruzada
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

-Averiguar otro tipo de cross_validator
-

In [0]:
# Entrenar el modelo con validación cruzada
cv_model = crossval.fit(train_data)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
# Predecir en los datos de prueba
test_results = cv_model.transform(test_data)

# Evaluar el modelo
rmse = evaluator.evaluate(test_results)
r2_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2 = r2_evaluator.evaluate(test_results)

# Mostrar los resultados de la evaluación
print(f"RMSE: {rmse}")
print(f"R²: {r2}")


RMSE: 343.98868608136524
R²: 0.023475009831276594


Este código implementa un pipeline de machine learning en PySpark para entrenar un modelo de Gradient Boosted Trees (GBT) con validación cruzada y ajustar los hiperparámetros. Vamos a desglosar cada sección para que comprendas qué está haciendo y cómo impacta en el proceso de construcción del modelo.

1. Crear una sesión de Spark y cargar datos
python
Copiar código
spark = SparkSession.builder.appName("ML_Model").getOrCreate()
df = spark.read.csv("/FileStore/tables/ecommerce_data_cleaned.csv", header=True, inferSchema=True)
Aquí se crea una sesión de Spark (el entorno para ejecutar operaciones distribuidas) y se carga el archivo CSV que contiene los datos limpios de comercio electrónico en un DataFrame llamado df.

2. Preparación de la etiqueta (target)
python
Copiar código
df_ml = df.withColumn("label", col("Revenue"))
El código crea una nueva columna llamada label a partir de la columna Revenue, que es la variable objetivo (lo que el modelo intentará predecir). Esta columna se utiliza en el modelo como la "etiqueta" en un problema de regresión.

3. Transformaciones de las características (features)
python
Copiar código
indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
encoder = OneHotEncoder(inputCol="CountryIndex", outputCol="CountryVec")
assembler = VectorAssembler(inputCols=["Quantity", "UnitPrice", "Month", "CountryVec"], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
Esta sección utiliza varias transformaciones para preparar los datos:

StringIndexer: Convierte la columna categórica Country en un índice numérico (CountryIndex).
OneHotEncoder: Convierte el índice numérico de Country en una representación de one-hot encoding (CountryVec), creando un vector binario para cada categoría.
VectorAssembler: Combina varias columnas numéricas como Quantity, UnitPrice, Month y CountryVec en una sola columna features, que el modelo puede usar.
StandardScaler: Normaliza los valores de las características para que todas estén en la misma escala, lo que es importante en modelos como GBT.
4. Pipeline de transformación
python
Copiar código
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler])
df_prepared = pipeline.fit(df_ml).transform(df_ml)
Este código construye un pipeline con las transformaciones anteriores (indexación, codificación, ensamblaje y escalado). El pipeline asegura que todas las etapas de preprocesamiento se ejecuten de manera secuencial. Luego, se transforma el DataFrame original para generar df_prepared, que contiene las columnas features y label.

5. División en conjunto de entrenamiento y prueba
python
Copiar código
train_data, test_data = df_prepared.randomSplit([0.8, 0.2], seed=42)
Se dividen los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%), lo cual es necesario para evaluar el rendimiento del modelo en datos no vistos.

6. Definición y entrenamiento del modelo
python
Copiar código
gbt = GBTRegressor(featuresCol="scaledFeatures", labelCol="label")
Se define un regresor GBT (Gradient Boosted Trees) para predecir la columna label usando las características scaledFeatures. GBT es un modelo potente que entrena una serie de árboles de decisión secuenciales, optimizando el error en cada paso.

7. Validación cruzada y ajuste de hiperparámetros
python
Copiar código
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.stepSize, [0.1, 0.05]) \
    .build()

crossval = CrossValidator(estimator=gbt, 
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator, 
                          numFolds=5)
Aquí se configura una validación cruzada de 5 pliegues con un grid de hiperparámetros para probar diferentes combinaciones de los parámetros maxDepth, maxIter y stepSize. La validación cruzada ajusta el modelo en diferentes subconjuntos de los datos para encontrar la combinación óptima de estos hiperparámetros.

maxDepth: Profundidad máxima de los árboles.
maxIter: Número de iteraciones.
stepSize: Tasa de aprendizaje.
8. Evaluación del modelo
python
Copiar código
test_results = cv_model.transform(test_data)
rmse = evaluator.evaluate(test_results)
r2 = r2_evaluator.evaluate(test_results)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")
Se hace una predicción en los datos de prueba con el mejor modelo encontrado por la validación cruzada. Luego, se evalúan los resultados usando dos métricas:

RMSE (Error cuadrático medio): mide el error promedio en las predicciones.
R² (Coeficiente de determinación): mide qué tan bien el modelo se ajusta a los datos.
Conclusión
Este código realiza un flujo completo de preparación de datos, creación de un pipeline de procesamiento, entrenamiento de un modelo de GBT, ajuste de hiperparámetros con validación cruzada, y evaluación del modelo en los datos de prueba. Si los valores de RMSE y R² no son satisfactorios, se podrían probar otras características, ajustar mejor los hiperparámetros o considerar otros modelos más adecuados para el conjunto de datos.